# Γενετικά δίκτυα

Τα Επαναλαμβανόμενα Νευρωνικά Δίκτυα (RNNs) και οι παραλλαγές τους με πύλες, όπως τα Κύτταρα Μνήμης Μακράς και Βραχείας Διάρκειας (LSTMs) και οι Μονάδες Επαναλαμβανόμενης Πύλης (GRUs), παρέχουν έναν μηχανισμό για τη μοντελοποίηση γλώσσας, δηλαδή μπορούν να μάθουν τη σειρά των λέξεων και να προβλέψουν την επόμενη λέξη σε μια ακολουθία. Αυτό μας επιτρέπει να χρησιμοποιούμε τα RNNs για **γενετικές εργασίες**, όπως η απλή δημιουργία κειμένου, η μηχανική μετάφραση και ακόμη και η περιγραφή εικόνων.

Στην αρχιτεκτονική RNN που συζητήσαμε στην προηγούμενη ενότητα, κάθε μονάδα RNN παρήγαγε την επόμενη κρυφή κατάσταση ως έξοδο. Ωστόσο, μπορούμε επίσης να προσθέσουμε μια άλλη έξοδο σε κάθε επαναλαμβανόμενη μονάδα, η οποία θα μας επιτρέπει να παράγουμε μια **ακολουθία** (που έχει το ίδιο μήκος με την αρχική ακολουθία). Επιπλέον, μπορούμε να χρησιμοποιήσουμε μονάδες RNN που δεν δέχονται είσοδο σε κάθε βήμα, αλλά απλώς λαμβάνουν κάποιο αρχικό διανύσμα κατάστασης και στη συνέχεια παράγουν μια ακολουθία εξόδων.

Σε αυτό το σημειωματάριο, θα επικεντρωθούμε σε απλά γενετικά μοντέλα που μας βοηθούν να δημιουργούμε κείμενο. Για απλότητα, ας κατασκευάσουμε ένα **δίκτυο σε επίπεδο χαρακτήρων**, το οποίο δημιουργεί κείμενο γράμμα προς γράμμα. Κατά τη διάρκεια της εκπαίδευσης, πρέπει να πάρουμε κάποιο σώμα κειμένου και να το χωρίσουμε σε ακολουθίες γραμμάτων.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Δημιουργία λεξιλογίου χαρακτήρων

Για να δημιουργήσουμε ένα γενετικό δίκτυο σε επίπεδο χαρακτήρων, πρέπει να χωρίσουμε το κείμενο σε μεμονωμένους χαρακτήρες αντί για λέξεις. Η στρώση `TextVectorization` που χρησιμοποιούσαμε προηγουμένως δεν μπορεί να το κάνει αυτό, οπότε έχουμε δύο επιλογές:

* Να φορτώσουμε το κείμενο χειροκίνητα και να κάνουμε την τοκενοποίηση "με το χέρι", όπως στο [επίσημο παράδειγμα του Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Να χρησιμοποιήσουμε την κλάση `Tokenizer` για τοκενοποίηση σε επίπεδο χαρακτήρων.

Θα επιλέξουμε τη δεύτερη επιλογή. Η `Tokenizer` μπορεί επίσης να χρησιμοποιηθεί για τοκενοποίηση σε λέξεις, οπότε κάποιος μπορεί να μεταβεί εύκολα από τοκενοποίηση σε επίπεδο χαρακτήρων σε επίπεδο λέξεων.

Για να κάνουμε τοκενοποίηση σε επίπεδο χαρακτήρων, πρέπει να περάσουμε την παράμετρο `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Θέλουμε επίσης να χρησιμοποιήσουμε ένα ειδικό σύμβολο για να δηλώσουμε το **τέλος της ακολουθίας**, το οποίο θα ονομάσουμε `<eos>`. Ας το προσθέσουμε χειροκίνητα στο λεξιλόγιο:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Τώρα, για να κωδικοποιήσουμε κείμενο σε ακολουθίες αριθμών, μπορούμε να χρησιμοποιήσουμε:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Εκπαίδευση ενός γενετικού RNN για τη δημιουργία τίτλων

Ο τρόπος με τον οποίο θα εκπαιδεύσουμε το RNN για να δημιουργεί τίτλους ειδήσεων είναι ο εξής. Σε κάθε βήμα, θα παίρνουμε έναν τίτλο, ο οποίος θα εισάγεται σε ένα RNN, και για κάθε χαρακτήρα εισόδου θα ζητάμε από το δίκτυο να παράγει τον επόμενο χαρακτήρα εξόδου:

![Εικόνα που δείχνει ένα παράδειγμα δημιουργίας της λέξης 'HELLO' από RNN.](../../../../../translated_images/el/rnn-generate.56c54afb52f9781d.webp)

Για τον τελευταίο χαρακτήρα της ακολουθίας μας, θα ζητάμε από το δίκτυο να παράγει το token `<eos>`.

Η κύρια διαφορά του γενετικού RNN που χρησιμοποιούμε εδώ είναι ότι θα παίρνουμε έξοδο από κάθε βήμα του RNN, και όχι μόνο από το τελικό κελί. Αυτό μπορεί να επιτευχθεί καθορίζοντας την παράμετρο `return_sequences` στο κελί του RNN.

Έτσι, κατά τη διάρκεια της εκπαίδευσης, η είσοδος στο δίκτυο θα είναι μια ακολουθία κωδικοποιημένων χαρακτήρων συγκεκριμένου μήκους, και η έξοδος θα είναι μια ακολουθία του ίδιου μήκους, αλλά μετατοπισμένη κατά ένα στοιχείο και τερματισμένη με `<eos>`. Το minibatch θα αποτελείται από αρκετές τέτοιες ακολουθίες, και θα χρειαστεί να χρησιμοποιήσουμε **padding** για να ευθυγραμμίσουμε όλες τις ακολουθίες.

Ας δημιουργήσουμε συναρτήσεις που θα μετασχηματίσουν το dataset για εμάς. Επειδή θέλουμε να προσθέσουμε padding στις ακολουθίες σε επίπεδο minibatch, θα ομαδοποιήσουμε πρώτα το dataset καλώντας `.batch()`, και στη συνέχεια θα χρησιμοποιήσουμε `map` για να κάνουμε τον μετασχηματισμό. Έτσι, η συνάρτηση μετασχηματισμού θα παίρνει ολόκληρο το minibatch ως παράμετρο:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Μερικά σημαντικά πράγματα που κάνουμε εδώ:
* Πρώτα εξάγουμε το πραγματικό κείμενο από το string tensor
* Το `text_to_sequences` μετατρέπει τη λίστα των συμβολοσειρών σε μια λίστα από ακέραια tensors
* Το `pad_sequences` προσθέτει padding σε αυτά τα tensors μέχρι το μέγιστο μήκος τους
* Τέλος, κάνουμε one-hot κωδικοποίηση όλων των χαρακτήρων, καθώς και τη μετατόπιση και την προσθήκη του `<eos>`. Σύντομα θα δούμε γιατί χρειαζόμαστε τους χαρακτήρες σε μορφή one-hot κωδικοποίησης

Ωστόσο, αυτή η συνάρτηση είναι **Pythonic**, δηλαδή δεν μπορεί να μεταφραστεί αυτόματα σε υπολογιστικό γράφημα του Tensorflow. Θα έχουμε σφάλματα αν προσπαθήσουμε να χρησιμοποιήσουμε αυτή τη συνάρτηση απευθείας στη συνάρτηση `Dataset.map`. Πρέπει να περικλείσουμε αυτήν την Pythonic κλήση χρησιμοποιώντας το wrapper `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Η διάκριση μεταξύ των Pythonic και Tensorflow συναρτήσεων μετασχηματισμού μπορεί να φαίνεται αρκετά περίπλοκη, και ίσως αναρωτιέστε γιατί δεν μετασχηματίζουμε το dataset χρησιμοποιώντας τις τυπικές συναρτήσεις της Python πριν το περάσουμε στη `fit`. Παρόλο που αυτό μπορεί σίγουρα να γίνει, η χρήση του `Dataset.map` έχει ένα μεγάλο πλεονέκτημα, καθώς ο αγωγός μετασχηματισμού δεδομένων εκτελείται χρησιμοποιώντας το υπολογιστικό γράφημα του Tensorflow, το οποίο αξιοποιεί τους υπολογισμούς της GPU και ελαχιστοποιεί την ανάγκη μεταφοράς δεδομένων μεταξύ CPU/GPU.

Τώρα μπορούμε να κατασκευάσουμε το δίκτυο γεννήτριας και να ξεκινήσουμε την εκπαίδευση. Μπορεί να βασίζεται σε οποιοδήποτε επαναληπτικό κύτταρο που συζητήσαμε στην προηγούμενη ενότητα (απλό, LSTM ή GRU). Στο παράδειγμά μας θα χρησιμοποιήσουμε LSTM.

Επειδή το δίκτυο λαμβάνει χαρακτήρες ως είσοδο, και το μέγεθος του λεξιλογίου είναι αρκετά μικρό, δεν χρειαζόμαστε στρώμα ενσωμάτωσης, η είσοδος κωδικοποιημένη σε one-hot μπορεί να περάσει απευθείας στο κύτταρο LSTM. Το στρώμα εξόδου θα είναι ένας ταξινομητής `Dense` που θα μετατρέπει την έξοδο του LSTM σε αριθμούς κωδικοποιημένων one-hot tokens.

Επιπλέον, επειδή ασχολούμαστε με ακολουθίες μεταβλητού μήκους, μπορούμε να χρησιμοποιήσουμε το στρώμα `Masking` για να δημιουργήσουμε μια μάσκα που θα αγνοεί το συμπληρωμένο μέρος της συμβολοσειράς. Αυτό δεν είναι αυστηρά απαραίτητο, καθώς δεν μας ενδιαφέρει ιδιαίτερα οτιδήποτε υπερβαίνει το token `<eos>`, αλλά θα το χρησιμοποιήσουμε για να αποκτήσουμε κάποια εμπειρία με αυτόν τον τύπο στρώματος. Το `input_shape` θα είναι `(None, vocab_size)`, όπου το `None` υποδεικνύει την ακολουθία μεταβλητού μήκους, και το σχήμα εξόδου είναι επίσης `(None, vocab_size)`, όπως μπορείτε να δείτε από το `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Δημιουργία αποτελέσματος

Τώρα που έχουμε εκπαιδεύσει το μοντέλο, θέλουμε να το χρησιμοποιήσουμε για να δημιουργήσουμε κάποιο αποτέλεσμα. Πρώτα απ' όλα, χρειαζόμαστε έναν τρόπο να αποκωδικοποιήσουμε κείμενο που αναπαρίσταται από μια ακολουθία αριθμών συμβόλων. Για να το κάνουμε αυτό, θα μπορούσαμε να χρησιμοποιήσουμε τη συνάρτηση `tokenizer.sequences_to_texts`. Ωστόσο, αυτή δεν λειτουργεί καλά με την τοκενoποίηση σε επίπεδο χαρακτήρων. Επομένως, θα πάρουμε ένα λεξικό συμβόλων από τον tokenizer (ονομάζεται `word_index`), θα δημιουργήσουμε έναν αντίστροφο χάρτη και θα γράψουμε τη δική μας συνάρτηση αποκωδικοποίησης:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Τώρα, ας προχωρήσουμε στη δημιουργία. Ξεκινάμε με μια συμβολοσειρά `start`, την κωδικοποιούμε σε μια ακολουθία `inp`, και σε κάθε βήμα καλούμε το δίκτυό μας για να προβλέψει τον επόμενο χαρακτήρα.

Η έξοδος του δικτύου `out` είναι ένας διάνυσμα με `vocab_size` στοιχεία που αντιπροσωπεύουν τις πιθανότητες κάθε συμβόλου, και μπορούμε να βρούμε τον αριθμό του πιο πιθανό συμβόλου χρησιμοποιώντας το `argmax`. Στη συνέχεια, προσθέτουμε αυτόν τον χαρακτήρα στη λίστα των παραγόμενων συμβόλων και συνεχίζουμε τη δημιουργία. Αυτή η διαδικασία παραγωγής ενός χαρακτήρα επαναλαμβάνεται `size` φορές για να παραχθεί ο απαιτούμενος αριθμός χαρακτήρων, και τερματίζουμε νωρίτερα όταν συναντηθεί το `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Δειγματοληψία εξόδου κατά τη διάρκεια της εκπαίδευσης

Επειδή δεν έχουμε χρήσιμες μετρικές όπως η *ακρίβεια*, ο μόνος τρόπος να δούμε ότι το μοντέλο μας βελτιώνεται είναι μέσω της **δειγματοληψίας** παραγόμενων συμβολοσειρών κατά τη διάρκεια της εκπαίδευσης. Για να το κάνουμε αυτό, θα χρησιμοποιήσουμε **callbacks**, δηλαδή συναρτήσεις που μπορούμε να περάσουμε στη συνάρτηση `fit` και οι οποίες θα καλούνται περιοδικά κατά τη διάρκεια της εκπαίδευσης.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Αυτό το παράδειγμα ήδη παράγει αρκετά καλό κείμενο, αλλά μπορεί να βελτιωθεί περαιτέρω με διάφορους τρόπους:

* **Περισσότερο κείμενο**. Έχουμε χρησιμοποιήσει μόνο τίτλους για την εργασία μας, αλλά ίσως θέλετε να πειραματιστείτε με πλήρες κείμενο. Θυμηθείτε ότι τα RNNs δεν είναι ιδιαίτερα καλά στη διαχείριση μεγάλων ακολουθιών, οπότε έχει νόημα είτε να τα χωρίσετε σε μικρότερες προτάσεις, είτε να εκπαιδεύετε πάντα σε σταθερό μήκος ακολουθίας με κάποια προκαθορισμένη τιμή `num_chars` (π.χ. 256). Μπορείτε να δοκιμάσετε να τροποποιήσετε το παραπάνω παράδειγμα σε τέτοια αρχιτεκτονική, χρησιμοποιώντας [το επίσημο tutorial του Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) ως έμπνευση.

* **Πολυεπίπεδο LSTM**. Έχει νόημα να δοκιμάσετε 2 ή 3 επίπεδα κυττάρων LSTM. Όπως αναφέραμε στην προηγούμενη ενότητα, κάθε επίπεδο LSTM εξάγει συγκεκριμένα μοτίβα από το κείμενο, και στην περίπτωση του γεννήτορα σε επίπεδο χαρακτήρων μπορούμε να περιμένουμε ότι το χαμηλότερο επίπεδο LSTM θα είναι υπεύθυνο για την εξαγωγή συλλαβών, ενώ τα υψηλότερα επίπεδα - για λέξεις και συνδυασμούς λέξεων. Αυτό μπορεί να υλοποιηθεί απλά περνώντας την παράμετρο αριθμού επιπέδων στον κατασκευαστή του LSTM.

* Μπορείτε επίσης να πειραματιστείτε με **μονάδες GRU** και να δείτε ποιες αποδίδουν καλύτερα, καθώς και με **διαφορετικά μεγέθη κρυφών επιπέδων**. Πολύ μεγάλο κρυφό επίπεδο μπορεί να οδηγήσει σε υπερπροσαρμογή (π.χ. το δίκτυο θα μάθει ακριβώς το κείμενο), ενώ μικρότερο μέγεθος μπορεί να μην παράγει καλό αποτέλεσμα.


## Δημιουργία μαλακού κειμένου και θερμοκρασία

Στον προηγούμενο ορισμό της `generate`, πάντα επιλέγαμε τον χαρακτήρα με την υψηλότερη πιθανότητα ως τον επόμενο χαρακτήρα στο παραγόμενο κείμενο. Αυτό είχε ως αποτέλεσμα το κείμενο να "επαναλαμβάνεται" συχνά μεταξύ των ίδιων ακολουθιών χαρακτήρων ξανά και ξανά, όπως στο παρακάτω παράδειγμα:
```
today of the second the company and a second the company ...
```

Ωστόσο, αν κοιτάξουμε την κατανομή πιθανοτήτων για τον επόμενο χαρακτήρα, μπορεί να παρατηρήσουμε ότι η διαφορά μεταξύ των υψηλότερων πιθανοτήτων δεν είναι μεγάλη, π.χ. ένας χαρακτήρας μπορεί να έχει πιθανότητα 0.2, ενώ ένας άλλος 0.19, κ.ο.κ. Για παράδειγμα, όταν ψάχνουμε τον επόμενο χαρακτήρα στη σειρά '*play*', ο επόμενος χαρακτήρας μπορεί εξίσου να είναι είτε κενό, είτε **e** (όπως στη λέξη *player*).

Αυτό μας οδηγεί στο συμπέρασμα ότι δεν είναι πάντα "δίκαιο" να επιλέγουμε τον χαρακτήρα με την υψηλότερη πιθανότητα, καθώς η επιλογή του δεύτερου υψηλότερου μπορεί να οδηγήσει επίσης σε ουσιαστικό κείμενο. Είναι πιο σοφό να **δειγματοληπτούμε** χαρακτήρες από την κατανομή πιθανοτήτων που δίνεται από την έξοδο του δικτύου.

Αυτή η δειγματοληψία μπορεί να γίνει χρησιμοποιώντας τη συνάρτηση `np.multinomial`, η οποία υλοποιεί την αποκαλούμενη **πολυωνυμική κατανομή**. Μια συνάρτηση που υλοποιεί αυτή τη **μαλακή** δημιουργία κειμένου ορίζεται παρακάτω:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Έχουμε εισαγάγει μία ακόμη παράμετρο που ονομάζεται **θερμοκρασία**, η οποία χρησιμοποιείται για να υποδείξει πόσο αυστηρά πρέπει να ακολουθούμε την υψηλότερη πιθανότητα. Αν η θερμοκρασία είναι 1.0, κάνουμε δίκαιη δειγματοληψία πολυωνύμου, και όταν η θερμοκρασία τείνει στο άπειρο - όλες οι πιθανότητες γίνονται ίσες, και επιλέγουμε τυχαία τον επόμενο χαρακτήρα. Στο παρακάτω παράδειγμα μπορούμε να παρατηρήσουμε ότι το κείμενο γίνεται χωρίς νόημα όταν αυξάνουμε υπερβολικά τη θερμοκρασία, και μοιάζει με "κυκλικό" κείμενο που παράγεται δύσκολα όταν πλησιάζει το 0.



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
